# Opdracht 3 Soort van YOLO

YOLO
je hebt een grid en bepaald wat is de kans dat dit celletje de centrale cel is van bijvoorbeedl een plaatsje van een hond. Het model doet een voorspelling. YOLO is een pytorch ding. we gebruiken het model dat al gebruikt is om images te detecteren. Kijken waar er bounding boxes zijn. missende data, de totale oppervlak van het plaatjem hoeveel wordt opgevuld door tegels?
Oppervlak keer 250? en dan krijg je de opvulling, check of dit correct is. per plaatje breedte en hoogte bepalen. Per afbeelding een bestand maken voor de labels directory. Coords-idc.txt is een plaatje en de bounding box van tumoren. linksboven naar rechtsonder. 

### Laad libraries

In [ ]:
import os
import pandas as pd
import shutil
import random

### Data omzetten naar YOLO-formaat

In [ ]:
# Functie om annotaties om te zetten naar YOLO-formaat
def convert_to_yolo_format(coords_file, images_folder, output_folder):
    """
    Converteert annotaties van een CSV-bestand naar YOLO-formaat.

    Parameters:
    - coords_file (str): Pad naar het CSV-bestand met annotaties.
    - images_folder (str): Map met afbeeldingsbestanden.
    - output_folder (str): Map om de YOLO-annotaties op te slaan.

    Returns:
    None
    """
    # Maak output map aan als die nog niet bestaat
    os.makedirs(output_folder, exist_ok=True)

    # Laad annotaties
    coords = pd.read_csv(coords_file, header=None, names=["image_id", "xmin", "ymin", "xmax", "ymax"])

    for image_id in coords["image_id"].unique():
        # Selecteer alle annotaties voor een afbeelding
        image_coords = coords[coords["image_id"] == image_id]
        
        # Bereken de afmetingen van de afbeelding
        image_path = os.path.join(images_folder, f"{image_id}.jpeg")
        if not os.path.exists(image_path):
            print(f"Afbeelding {image_path} niet gevonden.")
            continue
        
        from PIL import Image
        with Image.open(image_path) as img:
            width, height = img.size

        # Genereer YOLO-annotaties
        yolo_annotations = []
        for _, row in image_coords.iterrows():
            x_center = ((row["xmin"] + row["xmax"]) / 2) / width
            y_center = ((row["ymin"] + row["ymax"]) / 2) / height
            bbox_width = (row["xmax"] - row["xmin"]) / width
            bbox_height = (row["ymax"] - row["ymin"]) / height
            yolo_annotations.append(f"1 {x_center} {y_center} {bbox_width} {bbox_height}")

        # Schrijf annotaties naar bestand
        annotation_path = os.path.join(output_folder, f"{image_id}.txt")
        with open(annotation_path, "w") as f:
            f.write("\n".join(yolo_annotations))

# Bestandspaden
coords_file = "../data/coords-idc.txt"
images_folder = "../data/complete_images"
output_folder = "../yolo_labels"
convert_to_yolo_format(coords_file, images_folder, output_folder)

### Directory-structuur creëren

In [ ]:
def create_dataset_structure(images_folder, labels_folder, output_folder, train_ratio=0.8):
    """
    Creëert een gestructureerde dataset met een training- en validatieset.

    Parameters:
    - images_folder (str): Map met afbeeldingsbestanden.
    - labels_folder (str): Map met YOLO-annotaties.
    - output_folder (str): Doelmap voor gestructureerde dataset.
    - train_ratio (float): Ratio van training-data tot validatie-data.

    Returns:
    None
    """
    # Maak mapstructuur
    for split in ["train", "val"]:
        os.makedirs(os.path.join(output_folder, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_folder, "labels", split), exist_ok=True)

    # Verdeel data in training- en validatiesets
    image_files = [f for f in os.listdir(images_folder) if f.endswith(".jpeg")]
    random.shuffle(image_files)
    train_split = int(len(image_files) * train_ratio)

    splits = {
        "train": image_files[:train_split],
        "val": image_files[train_split:]
    }

    for split, files in splits.items():
        for image_file in files:
            # Verplaats afbeelding
            src_image = os.path.join(images_folder, image_file)
            dest_image = os.path.join(output_folder, "images", split, image_file)
            shutil.copy(src_image, dest_image)

            # Verplaats corresponderend label
            label_file = image_file.replace(".jpeg", ".txt")
            src_label = os.path.join(labels_folder, label_file)
            dest_label = os.path.join(output_folder, "labels", split, label_file)
            if os.path.exists(src_label):
                shutil.copy(src_label, dest_label)

images_folder = "../data/complete_images"
labels_folder = "../yolo_labels"
output_folder = "../dataset"
create_dataset_structure(images_folder, labels_folder, output_folder)

### YOLO-configuratiebestand

In [4]:
with open("../dataset.yaml", "w") as f:
    f.write(
        f"""train: {os.path.abspath(output_folder)}/images/train
val: {os.path.abspath(output_folder)}/images/val

nc: 1
names: ['tumor']
"""
    )

### YOLOv5 trainen

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov5n.pt")
model.train(data="dataset.yaml", epochs=50, single_cls=True)

### Voorspellingen doen

In [ ]:
def make_predictions(model_path, test_images_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    model = YOLO(model_path)
    results = model.predict(source=test_images_folder, save=True)
    return results

make_predictions("runs/detect/train/weights/best.pt", "test_images", "predictions")